Objective:
- Show values over time
- Calc return

Input:

  - Trade (time, symbol, size, ->value)
  - Cash flow (time, amount, type -> inflow, outflow, tax, interest)
  - Price (time, symbol, qfq_price,bfq_price)
     - source: http://tushare.org/trading.html#id3

Processing:
  - Trade + Cash => Position
  - Position * price => Value
  - Value/value => return

Issues:
    - Price fq: stock split, dividend
    - Cash in/out: not affect performance    

Pyfolio data definition:

    returns : pd.Series
        Daily returns of the strategy, noncumulative.
         - Time series with decimal returns.
         - Example:
            2015-07-16    -0.012143
            2015-07-17    0.045350
            2015-07-20    0.030957
            2015-07-21    0.004902
    positions : pd.DataFrame, optional
        Daily net position values.
         - Time series of dollar amount invested in each position and cash.
         - Days where stocks are not held can be represented by 0 or NaN.
         - Non-working capital is labelled 'cash'
         - Example:
            index         'AAPL'         'MSFT'          cash
            2004-01-09    13939.3800     -14012.9930     711.5585
            2004-01-12    14492.6300     -14624.8700     27.1821
            2004-01-13    -13853.2800    13653.6400      -43.6375
    transactions : pd.DataFrame, optional
        Executed trade volumes and fill prices.
        - One row per trade.
        - Trades on different names that occur at the
          same time will have identical indicies.
        - Example:
            index                  amount   price    symbol
            2004-01-09 12:18:01    483      324.12   'AAPL'
            2004-01-09 12:18:01    122      83.10    'MSFT'
            2004-01-13 14:12:23    -75      340.43   'AAPL'
    market_data : pd.Panel, optional
        Panel with items axis of 'price' and 'volume' DataFrames.
        The major and minor axes should match those of the
        the passed positions DataFrame (same dates and symbols).


In [5]:
import pandas as pd
import numpy as np
import pyfolio as pf
import matplotlib.pyplot as plt
import tushare as ts

In [5]:

longer_ts = pd.Series(np.random.randn(10),
                      index=pd.date_range('1/1/2000', periods=10))
longer_ts
# longer_ts['2001']

2000-01-01    0.135392
2000-01-02   -1.383609
2000-01-03   -0.785236
2000-01-04   -1.531916
2000-01-05    0.454635
2000-01-06    0.924728
2000-01-07   -0.687800
2000-01-08   -0.334572
2000-01-09   -0.918795
2000-01-10    0.647721
Freq: D, dtype: float64

In [9]:
longer_ts['2000-01-01']

0.13539175094809736

In [70]:
txn=pd.read_csv('txn.csv',sep=',',parse_dates=[0],skiprows=0,dtype={'index':str,'amount':np.int16,'price':np.float,'symbol':str})

In [71]:
txn

,index,amount,price,symbol
0,2018-07-17 12:18:01,100,750.0,'600519'
1,2019-04-01 14:12:23,-100,860.0,'600519'


In [48]:
txn.index

RangeIndex(start=0, stop=2, step=1)

In [49]:
txn.dtypes

index     datetime64[ns]
amount             int16
price            float64
symbol            object
dtype: object

In [34]:
price=pd.read_csv('MKTDATA.csv',sep=' ',parse_dates=[0],skiprows=0,\
                  dtype={'date':str, 'high':float,'open':float,'close':float,\
                       'low':float,'volume':float,'amount':float})

In [35]:
price.index

RangeIndex(start=0, stop=10, step=1)

In [36]:
price.dtypes

date           datetime64[ns]
Unnamed: 1            float64
Unnamed: 2            float64
Unnamed: 3            float64
Unnamed: 4            float64
Unnamed: 5            float64
Unnamed: 6            float64
Unnamed: 7            float64
Unnamed: 8            float64
Unnamed: 9            float64
Unnamed: 10           float64
open                  float64
Unnamed: 12           float64
Unnamed: 13           float64
high                  float64
Unnamed: 15           float64
close                 float64
Unnamed: 17           float64
Unnamed: 18           float64
Unnamed: 19           float64
low                   float64
Unnamed: 21           float64
Unnamed: 22           float64
Unnamed: 23           float64
Unnamed: 24           float64
volume                float64
Unnamed: 26           float64
Unnamed: 27           float64
Unnamed: 28           float64
Unnamed: 29           float64
Unnamed: 30           float64
amount                float64
dtype: object

In [26]:
Price=ts.get_h_data('600690',start='2015-01-01',end='2015-09-01',retry_count=10)

[Getting data:]#

In [27]:
Price

""


In [59]:
try:
    Price=ts.get_h_data('600690', start='2013-01-01', end='2015-03-16') #两个日期之间的前复权数据
except:
    print('sth wrong')

[Getting data:]#

In [61]:
Price['symbol']='600690'

In [62]:
Price.to_csv('mktdata600690.csv')

In [63]:
Price.tail()

,open,high,close,low,volume,price_change,p_change,ma5,ma10,ma20,v_ma5,v_ma10,v_ma20,symbol
date,,,,,,,,,,,,,,
2016-10-14,10.15,10.20,10.16,10.10,150256.12,0.00,0.00,10.242,10.242,10.242,149870.56,149870.56,149870.56,600690
2016-10-13,10.22,10.23,10.17,10.15,103950.10,-0.03,-0.29,10.263,10.263,10.263,149774.18,149774.18,149774.18,600690
2016-10-12,10.30,10.30,10.21,10.17,136781.16,-0.11,-1.07,10.293,10.293,10.293,165048.87,165048.87,165048.87,600690
2016-10-11,10.34,10.38,10.32,10.28,153272.67,-0.03,-0.29,10.335,10.335,10.335,179182.72,179182.72,179182.72,600690
2016-10-10,10.18,10.38,10.35,10.14,205092.77,0.21,2.07,10.350,10.350,10.350,205092.77,205092.77,205092.77,600690


In [45]:
Price.dtypes

open            float64
high            float64
close           float64
low             float64
volume          float64
price_change    float64
p_change        float64
ma5             float64
ma10            float64
ma20            float64
v_ma5           float64
v_ma10          float64
v_ma20          float64
dtype: object

for each symbol
    get initial offer date
    for every 3 years:
        get price

In [3]:
symbols=['000001','000651','000858','002001',\
         '002511','600036','600487','600518','600690','600789']

In [10]:
def get_price(symbols,start,end):
    if (start>end):
        raise 'Start cannot greater than end!'
        return None
    for symbol in symbols:
        if (start+3>end):
            price=ts.get_hist_data(symbol,start=str(start)+'-01-01',end=str(start+3)+'-01-01')
            price['symbol']=symbol
            price.to_csv('price.csv',mode='a+')
        else:
            i=start
            while (i+3<end):
                price=ts.get_hist_data(symbol,start=str(i)+'-01-01',end=str(i+3)+'-01-01')
                price['symbol']=symbol
                price.to_csv('price.csv',mode='a+')
                i=i+3
        

In [11]:
get_price(symbols,2010,2020)

In [34]:
mktdata=pd.read_csv('price.csv',sep=',')

In [35]:
mktdata.head()

,date,open,high,close,low,volume,price_change,p_change,ma5,ma10,ma20,v_ma5,v_ma10,v_ma20,symbol
0,date,open,high,close,low,volume,price_change,p_change,ma5,ma10,ma20,v_ma5,v_ma10,v_ma20,symbol
1,date,open,high,close,low,volume,price_change,p_change,ma5,ma10,ma20,v_ma5,v_ma10,v_ma20,symbol
2,2018-12-28,9.31,9.46,9.38,9.31,576604.0,0.1,1.08,9.344,9.623,9.98,543614.19,641599.88,599667.24,000001
3,2018-12-27,9.45,9.49,9.28,9.28,624593.25,-0.02,-0.21,9.358,9.702,10.029,628416.76,636606.99,600572.81,000001
4,2018-12-26,9.35,9.42,9.3,9.27,421140.59,-0.04,-0.43,9.444,9.813,10.075,701555.07,653232.18,593607.35,000001


In [44]:
mktdata=mktdata[mktdata.symbol in symbols]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [75]:
txn

,index,amount,price,symbol
0,2018-07-17 12:18:01,100,750.0,'600519'
1,2019-04-01 14:12:23,-100,860.0,'600519'


In [59]:
positions=pd.DataFrame(index=pd.date_range('9/1/2016', periods=1000),columns=["cash"])

In [74]:
positions.tail(10)

,cash,000001,000651,000858,002001,002511,600036,600487,600518,600690,600789
2019-05-19,1000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-05-20,1000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-05-21,1000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-05-22,1000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-05-23,1000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-05-24,1000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-05-25,1000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-05-26,1000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-05-27,1000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-05-28,1000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
positions['cash']=1000000.

In [68]:
for symbol in symbols:
    positions[symbol]=0.

In [81]:
for t in range(txn.size):
    print(txn[txn.iloc(t)])

KeyError: <pandas.core.indexing._iLocIndexer object at 0x7f4792e5ce10>